# Tema 5: Detección de líneas

En este cuaderno se desarrollan los ejemplos prácticos referentes al tema de expocisión, la intención es conocer algunas funcionalidades y herramientas que se utilizan con regularidad en procesos de visión computacional, específicamente para la detección de líneas.

**Objetivos**
* Implementar de manera práctica filtros en imágenes para la detección de líneas.
* Utilizar OpenCV como principal líbreria y conocer algunas de sus funciones.
* Analizar el proceso matemático que utilizan los filtros de detección de líneas.